In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import json

In [0]:
class Validate_ATL:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.data_json = {"Data":[]}    
  
  def logger(message):
    print(message)
    
  def set_query_string(self):
        query ='''with m(usecase_id, hist_retention) AS (SELECT USECASE_ID,hist_retention from com_us_alyt_ngebox.metadata_usecase),
s(SUGGEST_EXTERNAL_ID_VOD__C,NGEBox_Record_ID) AS (select p.SUGGEST_EXTERNAL_ID_VOD__C, p.NGEBox_Record_ID from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p 
left join com_us_lake.rep_suggestion_vod__c s on 
p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C 
where (s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) 
or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)),
t(Record_ID,account_ID,Territory_name) as (select distinct tb1.NGEBOX_RECORD_ID, tb1.account_vod__c,tb2.geo_cd 
  from com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed tb1
  join com_us_hub.ref_zip_terr tb2 on tb1.hcp_zip_code = tb2.zip_cd and tb2.curr_ind = 'Y' and tb2.level = 'Territory'
  join (select account_vod__c, explode(split(btrim(TERRITORY_VOD__C,';'),';')) as abc from com_us_alyt_omnichannel.aws_ib_ilay_rep_account_territory_loader_v) tb3 on tb1.account_vod__c = tb3.account_vod__c and tb2.geo_cd = tb3.abc
  left join com_us_lake.rep_suggestion_vod__c tb4 on tb1.SUGGEST_EXTERNAL_ID_VOD__C = tb4.SUGGESTION_EXTERNAL_ID_VOD__C
  where (tb4.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (tb4.SUGGESTION_EXTERNAL_ID_VOD__C is not null and tb4.DISMISSED_VOD__C = 0)
  order by tb1.NGEBOX_RECORD_ID,  tb2.geo_cd)
update com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed process_table
set process_table.Not_Sent_To_Veeva_Reason = (case  
  when ISNULL(process_table.Not_Sent_To_Veeva_Reason)= true 
  then 'Missing ATL in iEngage' 
  else concat(process_table.Not_Sent_To_Veeva_Reason, '|', 'Missing ATL in iEngage' ) 
  end) 
where CAST(process_table.created_date as DATE) >= date_add(current_date, -(select m.hist_retention from m where m.usecase_id = usecase_id))
and process_table.account_vod__c not in (select account_ID from t where t.Record_ID = process_table.NGEBOX_RECORD_ID )
and process_table.SUGGEST_EXTERNAL_ID_VOD__C in (select SUGGEST_EXTERNAL_ID_VOD__C from s)
'''
        self.query = query  

  def run_query_and_set_sqldf(self):
    query_list = self.query.split(":")
    self.sql_df = []
    for curr_query in query_list:
      if curr_query.strip() == "":
        continue
      sql_df = sqlContext.sql(curr_query).collect() # type(sql_df) -> dict
      self.sql_df = self.sql_df + sql_df
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    nb_json = {"data":sql_jsonparse} # type(metadata_json) -> dict
    self.nb_json = nb_json
    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.nb_json
      }
    }
    self.response = resp

In [0]:
nb_obj = Validate_ATL()
nb_obj.set_query_string()
nb_obj.run_query_and_set_sqldf()
nb_obj.format_sqldf_to_json()
nb_obj.construct_microservice_response()
dbutils.notebook.exit(
  nb_obj.response
)

{'nge_response': {'status': 200, 'body': {'data': [{'num_affected_rows': 10}]}}}